# Intro

blah blaj

In [25]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, preprocessing, svm
import math


## Reading file from gzip

In [26]:
df = pd.read_csv('autos.csv.gz', sep=',', header=0, compression='gzip',encoding='cp1252')

## Cleaning data from outliers and dirty values

In [27]:
# Cleaning data
valid_models = df.dropna() #.loc[df['model'] != 'NaN']

#### Removing the duplicates
dedups = valid_models.drop_duplicates(['name','seller','offerType','price','abtest','vehicleType','yearOfRegistration'
                         ,'gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType'
                         ,'notRepairedDamage','postalCode'])

#### Removing the outliers
no_outliers = dedups[(df.yearOfRegistration <=  2016) 
      & (df.yearOfRegistration >= 1950) 
      & (df.price >= 500) 
      & (df.price <= 50000) 
      & (df.powerPS >= 30) 
      & (df.powerPS <= 500) 
      & (df.kilometer >= 1000) 
      & (df.kilometer <= 200000)]

#rel_cols['price'].unique()
#rel_cols['yearOfRegistration'].unique()
#rel_cols['gearbox'].unique()
#rel_cols['powerPS'].unique()
#rel_cols['model'].unique()
#rel_cols['kilometer'].unique()
#rel_cols['monthOfRegistration'].unique()
#rel_cols['brand'].unique()
#rel_cols['notRepairedDamage'].unique()


/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


## Selecting only relevant columns

In [28]:

# only relevant columns
rel_cols = no_outliers[['price'
                        ,'yearOfRegistration'
                        ,'gearbox'
                        ,'powerPS'
                        ,'model'
                        ,'kilometer'
                        ,'monthOfRegistration'
                        ,'brand'
                        ,'notRepairedDamage']]
rel_cols

,price,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,brand,notRepairedDamage
3,1500,2001,manuell,75,golf,150000,6,volkswagen,nein
4,3600,2008,manuell,69,fabia,90000,7,skoda,nein
5,650,1995,manuell,102,3er,150000,10,bmw,ja
6,2200,2004,manuell,109,2_reihe,150000,8,peugeot,nein
10,2000,2004,manuell,105,3_reihe,150000,12,mazda,nein
11,2799,2005,manuell,140,passat,150000,12,volkswagen,ja
14,17999,2011,manuell,190,navara,70000,3,nissan,nein
17,1750,2004,automatik,75,twingo,150000,2,renault,nein
18,7550,2007,manuell,136,c_max,150000,6,ford,nein
19,1850,2004,manuell,102,a_klasse,150000,1,mercedes_benz,nein


In [29]:

le_gearbox = preprocessing.LabelEncoder()
le_notRepairedDamage = preprocessing.LabelEncoder()
le_model = preprocessing.LabelEncoder()
le_brand = preprocessing.LabelEncoder()

le_gearbox.fit(rel_cols['gearbox'])
tr = le_gearbox.transform(rel_cols['gearbox']) 
rel_cols.loc[:, 'gearbox_feat'] = pd.Series(tr, index=rel_cols.index)

le_notRepairedDamage.fit(rel_cols['notRepairedDamage'])
tr = le_notRepairedDamage.transform(rel_cols['notRepairedDamage']) 
rel_cols.loc[:, 'notRepairedDamage_feat'] = pd.Series(tr, index=rel_cols.index)

le_model.fit(rel_cols['model'])
tr = le_model.transform(rel_cols['model']) 
rel_cols.loc[:, 'model_feat'] = pd.Series(tr, index=rel_cols.index)

le_brand.fit(rel_cols['brand'])
tr = le_brand.transform(rel_cols['brand']) 
rel_cols.loc[:, 'brand_feat'] = pd.Series(tr, index=rel_cols.index)

autos = rel_cols[ ['price'
                        ,'yearOfRegistration'
                        ,'gearbox_feat'
                        ,'powerPS'
                        ,'model_feat'
                        ,'kilometer'
                        ,'monthOfRegistration'
                        ,'brand_feat'
                        ,'notRepairedDamage_feat']]



#autos_norm = normalizer.transform(autos)
autos_norm = autos.values.astype(float) #scaler.transform(autos)

Y_norm = autos_norm[:,0]

#normalizer = preprocessing.Normalizer().fit(autos_norm[:,1:])
scaler = preprocessing.StandardScaler().fit(autos_norm[:,1:])
X_norm = scaler.transform(autos_norm[:,1:])

print(Y_norm.shape)
print(X_norm.shape)

(235151,)
(235151, 8)


## Linear Regression

In [30]:
linear = linear_model.LinearRegression()

# Percent of the X array to use as training set. This implies that the rest will be test set
training_set_size = .9

train_len = int(X_norm.shape[0] * training_set_size)
X_train = X_norm[:train_len]
Y_train = Y_norm[:train_len]
X_test = X_norm[train_len+1:]
Y_test = Y_norm[train_len+1:]

linear.fit(X_train,Y_train)


Y_predict = linear.predict(X_test)
print("Score: ", linear.score(X_test, Y_test) ) 

print("Mean squared error: %.3f"
      % np.mean((Y_predict - Y_test) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % linear.score(X_test, Y_test))

# 0.60166638059 not-normalized
# 0.81354026898039011 normalized

('Score: ', 0.6258692759262352)
Mean squared error: 18439905.368
Variance score: 0.63


### Testing on a sample

In [31]:
sample = [2015, 'manuell', 115, 'touran', 20000, 1, 'volkswagen', 'nein']

sample[1] = le_gearbox.transform(sample[1])
sample[3] = le_model.transform(sample[3])
sample[6] = le_brand.transform(sample[6]) 
sample[7] = le_notRepairedDamage.transform(sample[7]) 

sample = scaler.transform([sample])

s_predict = linear.predict(sample)

s_predict


array([ 16674.63766664])

# Other tests

In [32]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

ImportError: No module named model_selection